In [ ]:
# @title CaveXML Query Interface
# This script searches a CaveXML database for caves with specific properties

import xml.etree.ElementTree
import urllib.request
import httpimport
import re   
#from IPython.display import display


# Access XML database from internet
url = 'https://raw.githubusercontent.com/nschorgh/CaveXML/master/allcaves-database.xml'
site = urllib.request.urlopen(url)
data = site.read()
root = xml.etree.ElementTree.fromstring(data)

# import module from github
url2 = "https://raw.githubusercontent.com/nschorgh/CaveXML/master/Utilities/"
with httpimport.remote_repo(['cavexml'], url2):
   import cavexml

print('Data source:',url)
print('')

# Form fields
countryname = "None" #@param ["None", "United States", "Antarctica", "Argentinia", "Australia", "Austria", "Bosnia and Herzegovina", "Bulgaria", "Cameroon", "Canada", "Chile", "China", "Comoros", "Congo, Democratic Republic of the", "Croatia", "Ecuador", "Fiji", "Finland", "France", "Germany", "Greece", "Greenland", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Italy", "India", "Indonesia", "Iran", "Italy", "Japan", "Jordan", "Kazakhstan", "Kenya", "Korea, Republic of", "Madagascar", "Malaysia", "Mauritius", "Mexico", "Micronesia", "Mongolia", "Montenegro", "New Zealand", "North Macedonia", "Norway", "Poland", "Portugal", "Romania", "Russia", "Rwanda", "Samoa", "Saudi Arabia", "Serbia", "Slovakia", "Slovenia", "Spain", "Svalbard and Jan Mayen", "Sweden", "Switzerland", "Syria", "Tajikistan", "Tanzania", "Turkey", "United Kingdom", "Uzbekistan", "Vanuatu", "Viet Nam", "Moon", "Mars"]
#@markdown Enter cave name or partial cave name:
cavename = '' #@param {type:"string"}
#@markdown Select minimum altitude in meter:
altitude_above = 0 #@param {type:"slider", min:0, max:5000, step:50}
#@markdown Select minimum gallery length in kilometer:
longer_than = 0 #@param {type:"slider", min:0, max:25, step:0.2}
rocktype = "None" #@param ["None", "limestone", "dune limestone", "dolomite", "marble", "basalt", "dolerite", "granite", "gypsum", "ice", "lava", "magnesite", "mudstone", "quartzite", "sandstone", "soil", "tuff"]
cavetype = "lava tunnel" #@param ["None", "solution cave", "artifical cave", "boulder cave", "tectonic cave", "basalt", "glacier cave", "lava tunnel", "lava vent", "eolian cave", "sea cave", "piping cave", "frost weathering cave", "misc. type"]
contents = "permanent ice" #@param ["None", "permanent ice", "periodic ice", "extensive guano", "many bats", "occasional bats", "birds", "fish", "snakes", "trogloxenes", "accidental trogloxenes", "troglophiles", "troglobites", "charcoal", "paintings", "minerals", "lake(s)", "waterfall(s)", "tree roots", "perennially submerged", "intermittently submerged", "perennially part-submerged", "intermittently part-submerged"]
comment = '' #@param {type:"string"}
caveuse = "None" #@param ["None", "guided tourist cave", "self-guided tourist cave", "waste disposal", "habitation", "livestock shelter", "water source", "mine", "shrine", "temple"]

#@markdown If more than one criterion is selected, each will be required.

#@markdown Press play button on upper left to execute search.

if countryname != 'None':
  print('Search for caves in country:',countryname)

if len(cavename)>0:
  print('Search for cave with name:',cavename)

if altitude_above > 0:
  print('Search for caves at altitudes above',altitude_above,'m')

if longer_than > 0:
  print('Search for caves longer than',longer_than,'km')

if rocktype != 'None':
  print('Search for caves of rock type:',rocktype)

if cavetype != 'None':
  print('Search for caves of type:',cavetype)

if contents != 'None':
  print('Search for caves that contain:',contents)

if len(comment)>0:
  print('Search for caves here comments include:',comment)

if caveuse != 'None':
  print('Search for caves used as',caveuse)

print('')
print('Results of query:')


count = 0
count_match = 0

for item in root.findall('record'):
        
    count = count + 1  # count number of records


    # extract basic info from record

    con = item.find('country-name')
    if con is not None:
        con = con.text
        iso2 = cavexml.country2alpha2(con)

    acn = cavexml.get_one_cave_name(item)
    uid = cavexml.generate_unique_id(item)

    cid = item.find('cave-id')
    intcid = ''
    if cid is not None:
        if cid.text:
            intcid = iso2 + '-' + cid.text


    # compare with query entries
    
    if countryname != 'None':
      if countryname in con:
        pass
      else:
        continue

    if len(cavename)>0:  # something was entered in the query
      if cavename in acn:
          pass
      else: 
          continue

    if altitude_above>0:
      alt = item.findall('altitude') 
      minmax = cavexml.parse_AltitudeEntry(alt)
      try:
        alt = float(minmax[1]) # pick maximum
      except:
        alt = -9999   # no altitude entries
      if alt < altitude_above:
        continue

    if longer_than > 0:
      leng = item.find('length')
      if leng is None:
        continue
      lennumb, approx, qual = cavexml.parse_ExtendedUnsignedInteger(leng.text)
      if lennumb is None:
        continue
      if lennumb < 1000*longer_than:
        continue

    if rocktype != 'None':  # something was entered in the query
      rock = item.findall('rock-type')
      match = False
      print('rock type',rocktype)
      if rock:
        for i in range(0,len(rock)):
          if rock[i].text:
            if rock[i].text in rocktype:
              match = True
              break
      if match is False:
        continue

    if cavetype != 'None':   # something was entered in the query
      ctype = item.findall('cave-type')
      match = False
      if ctype:
        for i in range(0,len(ctype)):
          if ctype[i].text:
            if ctype[i].text in cavetype:
              match = True
      if match is False:
        continue

    if contents != 'None':  # something was entered in the query
      cont = item.findall('contents')
      match = False
      if cont:
        for i in range(0,len(cont)):
          if cont[i].text:
            s = cont[i].text
            s = re.sub("\s\s+", " ", s)  # collapse repeat (interior) spaces
            if s in contents:
              match = True
      if match is False:
        continue

    if len(comment)>0:
      comm = item.findall('comments')
      comm = cavexml.merge_elements(comm)
      if comment in comm:
        pass
      else:
        continue

    if caveuse != 'None':   # something was entered in the query
      cu = item.findall('cave-use')
      match = False
      if cu is None:
        continue
      for i in range(0,len(cu)):
        if cu[i].text:
          if cu[i].text in caveuse:
            match = True
      if match is False:
        continue

    # output search results

    #IPython.core.display.display(HTML('<a href=" ' + baseurl + uid + ' ">[link]</a>'))
    #IPython.display.display_html('<a href=" ' + baseurl + uid + ' ">[link]</a>', raw=True)

    print(iso2,acn,intcid,sep='  ')
    count_match += 1


# output summary
print('')
print('Number of records searched:', count)
print('Number of matching records:', count_match)
if count_match>0:
  print('Output columns are ISO-2 country code, cave name, and, if available, cave id.')
